In [ ]:
import  numpy  as np
import pandas as pd 
import re
import string
import pickle

In [ ]:
def remove_punctuations(input_text):
    for punctuation in string.punctuation:
        input_text = input_text.replace(punctuation, '')
    return input_text

In [ ]:
with open('C:\Users\CHATHUSHKA\Desktop\Sentiment_analysis_project\SAP_1\project\model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
with open('../SAP_1/project/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [ ]:
vocab = pd.read_csv('../SAP_1/project/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
def preprocessing(input_text):
    data = pd.DataFrame([input_text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace("\\d+", "", regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [ ]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [ ]:
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1154), dtype=float32)

In [ ]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [ ]:
input_txt = "awesome product. i love it !"
preprocessed_txt = preprocessing(input_txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'